In [ ]:
!pip install evaluate Pillow==9.0.0
!pip install pytesseract transformers datasets rouge-score nltk tensorboard py7zr --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls

drive  sample_data


In [ ]:
DATAPATH = "drive/MyDrive/CS4248 Team 19/translated_data/final_random_2k_de.csv"
MODELPATH = "drive/MyDrive/CS4248 Team 19/saved_models/flan-t5-large-4lang-cot/"
TOKENIZERPATH = "drive/MyDrive/CS4248 Team 19/saved_models/flan-t5-large-4lang-cot/"

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained(TOKENIZERPATH)

In [ ]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained(MODELPATH).to("cuda:0")

In [ ]:
import pandas as pd

random_de_df = pd.read_csv(DATAPATH).values.tolist()

In [ ]:
print (random_de_df[0][0])
print (random_de_df[0][1])
print (random_de_df[0][2])
print (random_de_df[0][3])
print (random_de_df[0][4])
print (random_de_df[0][5])

0
Eine Person mit braunen Haaren trägt ein blaues Hemd und lila Bandanna um ihren Hals mischt ein Design in eine runde, Ziegelsäule.
Die Person, die ein blaues Hemd trägt, meißelt auf einer Ziegelsäule.
entailment
Eine Person meißelt ein Design aus Ziegelsteinen.
german


In [ ]:
import torch
from sklearn.metrics import classification_report, confusion_matrix

updated_prompts_de = []
for record in random_de_df:
    prem = record[1]
    hypo = record[2]
    text = f"""Answer whether the premise entails or contradicts the hypothesis or if it's impossible to tell. Let us think step-by-step.
Premise: {prem}
Hypothesis: {hypo}"""
    updated_prompts_de.append(text)

In [ ]:
from pprint import pprint

print (updated_prompts_de[0])
print (updated_prompts_de[1])

Answer whether the premise entails or contradicts the hypothesis or if it's impossible to tell. Let us think step-by-step.
Premise: Eine Person mit braunen Haaren trägt ein blaues Hemd und lila Bandanna um ihren Hals mischt ein Design in eine runde, Ziegelsäule.
Hypothesis: Die Person, die ein blaues Hemd trägt, meißelt auf einer Ziegelsäule.
Answer whether the premise entails or contradicts the hypothesis or if it's impossible to tell. Let us think step-by-step.
Premise: Zwei Männer sitzen in einem Wohnzimmer, einer trägt kulturelle Kleidung und ein anderer in einem weißen Hemd und Shorts, betrachten auf ihrem Fahrrad und Moped.
Hypothesis: Zwei Männer in blauen Anzügen verkaufen Fahrräder und Mopeds.


In [ ]:
BS = 50
device = torch.device("cuda:0")

corrupt = []

mapper = {"neutral": 0, "contradiction": 1, "entailment": 2}
y_pred = []
y_test = [mapper[record[3]] for record in random_de_df]

all_pred = []
count = 0

for i in range(0, len(updated_prompts_de), BS):
    batch = updated_prompts_de[i:i+BS]
    inputs = tokenizer(batch, return_tensors="pt", padding=True)

    outputs = model.generate(
                    inputs['input_ids'].to(device),
                    attention_mask=inputs['attention_mask'].to(device),
                    min_length=1,
                    max_new_tokens=40,
                    num_beams=4
                )
    preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    for j, pred in enumerate(preds):
        pred = pred.split(".")
        pred = list(filter(lambda x : x != "", pred))
        pred = list(map(lambda x : x.strip(), pred))
        # print (pred)

        pred_expl = pred[0]
        pred_label = pred[-1].lower().split()[-1]

        if pred_label in ["neutral", "contradiction", "entailment"]:
            y_pred.append(mapper[pred_label])
            if count < 50:
                all_pred.append([
                    pred_label,
                    pred_expl
                ])
                count += 1
        else:
            corrupt.append([pred_label, pred_expl, i, j])

print (len(y_pred))
print (len(y_test))

1453
2000


In [ ]:
# pprint (corrupt)
import copy

bad_idx = []

for record in corrupt:
    add = record[-1] + record[-2]
    bad_idx.append(add)

bad_idx = set(bad_idx)
valid_ids = list(set(list(range(0, 2000))) - bad_idx)

y_test_ = []

for i in valid_ids:
    y_test_.append(y_test[i])

print (len(y_test_))    

1453


In [ ]:
print (len(y_test_), len(y_pred))

1453 1453


In [ ]:
cls_data = list(zip(y_test_, y_pred))       
cls_data = pd.DataFrame(cls_data, columns=["Actual", "Prediction"])
conf_matrix = confusion_matrix(y_test_, y_pred)
cls_report = classification_report(y_test_, y_pred)

In [ ]:
print (conf_matrix)

[[232  58 107]
 [177 204 118]
 [ 93  26 438]]


In [ ]:
print (cls_report)

              precision    recall  f1-score   support

           0       0.46      0.58      0.52       397
           1       0.71      0.41      0.52       499
           2       0.66      0.79      0.72       557

    accuracy                           0.60      1453
   macro avg       0.61      0.59      0.58      1453
weighted avg       0.62      0.60      0.59      1453



In [ ]:
print (len(all_pred))

to_save = []

for item in zip(random_de_df[:50], all_pred):
    idx, prem, hypo, label, expl, _ = item[0] # _ is the language [german]
    pred_label, pred_expl = item[1]

    to_save.append([
        prem,
        hypo,
        label,
        pred_label,
        expl,
        pred_expl
    ])

50


In [ ]:
to_save = pd.DataFrame(to_save, columns=["premise", "hypothesis", "label", "pred_label", "explanation", "pred_explanation"])
to_save.to_csv(f"drive/MyDrive/CS4248 Team 19/Findings/flan-t5-large-4lang-cot-german-eval-first50.csv")